 # REGEL TIL RETTELSE: Objekter med 'vertikalStedfæstelsesmetode' = Fotogrammetri eller Landmåling må ikke have udefinerede koter

In [629]:
import cx_Oracle
import pandas as pd
import sqlalchemy
import geopandas as gpd
from shapely import wkt
import os
import glob
import rasterio
from shapely.geometry import Point, LineString, Polygon
from sqlalchemy import create_engine, text as sql_text
from sqlalchemy.engine import create_engine
import shutil
from pyproj import CRS
from datetime import datetime, timezone, timedelta
import pytz

# Connect to Oracle

In [630]:
DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = '' #enter your username
PASSWORD = '' #enter your password
HOST = '' #enter the oracle db host url
PORT = 1521 # enter the oracle port number
SERVICE = '' # enter the oracle db service name
ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

connection = create_engine(ENGINE_PATH_WIN_AUTH, pool_size=200)


# Query to get all tabel names from GEODANMARK_60_NOHIST

In [631]:
# Get All Tabel Names From GEODANMARK_60_NOHIST
query = """SELECT view_name FROM all_views WHERE owner = 'GEODANMARK_60_NOHIST'"""

Tabels = pd.read_sql_query(con=connection.connect(), 
                                  sql=sql_text(query))

# Functions

##  Query Function

In [292]:
def Run_query(query, tabel):
    
    # Run Query Using Tabel Name and Query String
    
    GeoDanmark_objekt = pd.read_sql_query(con=connection.connect(), 
                                  sql=sql_text(query))
    
    
    
    GeoDanmark_objekt['geometri'] = GeoDanmark_objekt['geometri2'].apply(wkt.loads)
    
    GeoDanmark_objekt = gpd.GeoDataFrame(GeoDanmark_objekt, geometry=GeoDanmark_objekt['geometri'])
    
    GeoDanmark_objekt['tabel'] = tabel
    
    GeoDanmark_objekt['geom_type'] = GeoDanmark_objekt['geometry'].geom_type
    
    return GeoDanmark_objekt

    

## Process Data Functions 

### Points

In [644]:
def process_data_point(gdf):
    # Create an empty GeoDataFrame
    corrected_points_gdf = gpd.GeoDataFrame(columns=['geometry_corrected', 'id_lokalid', 'tabel', 'old_Z', 'idx']) 
    
    # Initialize the changes_made flag
    changes_made = False
    
    # Check if the GeoDataFrame contains Point geometry type
    if 'Point' in gdf['geometry'].geom_type.unique():
        
        for idx, (geom, id_lokalid, tabel) in enumerate(zip(gdf['geometry'], gdf['id_lokalid'], gdf['tabel'])):
            z_value = geom.z  # Extract the Z value from the Point geometry

            # Check if the Z value is 1.6
            if z_value == -999:
                
                if gdf['tabel'][0] in DTM_objects:
                    
                    # Retrieve a new Z value from the Load_DHM function
                    new_z_value = Load_DTM(Kommune_kode, geom.x, geom.y, id_lokalid, tabel)
                    
                
                else:
                    
                    # Retrieve a new Z value from the Load_DHM function
                    new_z_value = Load_DHM(Kommune_kode, geom.x, geom.y, id_lokalid, tabel)
                
                # Create a corrected Point geometry with the new Z value
                corrected_geom = Point(geom.x, geom.y, new_z_value)
                
                data = {
                    'geometry_corrected': [corrected_geom],
                    'id_lokalid': [id_lokalid],
                    'tabel': [tabel],
                    'old_Z': [z_value],
                    'idx'   : [idx]
                }
                
                # Concatenate the new data to the corrected_points_gdf GeoDataFrame
                corrected_points_gdf = pd.concat([corrected_points_gdf, pd.DataFrame(data)], ignore_index=True)
                
                changes_made = True  # Set the flag to True to indicate changes were made
                
    # Check if any changes were made
    if changes_made:
        
        
        gdf = pd.merge(gdf, corrected_points_gdf, on=['tabel', 'id_lokalid'], how='inner')
        
    
        
        print('corrected points', len(corrected_points_gdf))
        print('len gdf', len(gdf))
        
        return gdf
    else:
        print('corrected points', 0)
        return None  # No changes were made, return None
                

### LineStrings

In [697]:
def process_data_linestrings(gdf):
    # Initialize the changes_made flag
    changes_made = False
    
    # Create an empty GeoDataFrame
    corrected_linestring_gdf = gpd.GeoDataFrame(columns=['geometry_corrected', 'id_lokalid', 'tabel', 'old_Z', 'idx'])  
    
    # Check if the GeoDataFrame contains LineString or MultiLineString geometry types
    if 'LineString' in gdf['geometry'].geom_type.unique():
        for idx, row in gdf.iterrows():
            geometry = row['geometry']

            # Check if the geometry type is 'LineString'
            if geometry.geom_type == 'LineString':
                # Get a list of coordinates from the LineString geometry
                coords_list = list(geometry.coords)
                
                z_value_list = []
                
                
                if any(-999 in coord for coord in coords_list):
                    # Iterate over each coordinate in the list
                    for i, (x, y, z) in enumerate(coords_list):
                        # Check if the Z value of the coordinate is -999
                        
                        
                        if z == -999:
                            z_value_list.append((i, z))
                            
                            if gdf['tabel'][0] in DTM_objects:
                                # Replace the Z value with a new value obtained from Load_DHM function
                                new_z_value = Load_DTM(Kommune_kode, x, y, row['id_lokalid'], row['tabel'])
                            
                            else: 
                                # Replace the Z value with a new value obtained from Load_DHM function
                                new_z_value = Load_DHM(Kommune_kode, x, y, row['id_lokalid'], row['tabel'])
                                
                            #insert Coordinates into the coord_list at the right place using index    
                            coords_list[i] = (x, y, new_z_value)
                            changes_made = True  # Set the flag to True to indicate changes were made

                    # Create a new LineString with the modified coordinates
                    modified_linestring = LineString(coords_list)


                    data = {
                        'geometry_corrected': [modified_linestring],
                        'id_lokalid': [row['id_lokalid']],
                        'tabel': [row['tabel']],
                        'old_Z': [z_value_list],
                        'idx': [idx]
                    }

                    # Concatenate the new data to the corrected_linestring_gdf GeoDataFrame
                    corrected_linestring_gdf = pd.concat([corrected_linestring_gdf, pd.DataFrame(data)], ignore_index=True)

    elif 'MultiLineString' in gdf['geometry'].geom_type.unique():
        print('_________ MULTI-LINESTRINGS NOT ACCEPTED __________')
        return None

    # Check if any changes were made
    if changes_made:
        print('corrected linestrings', len(corrected_linestring_gdf))
        
        gdf = pd.merge(gdf, corrected_linestring_gdf, on=['tabel', 'id_lokalid'], how='inner')
        
        print('len gdf', len(gdf))
        
        return gdf
    else:
        print('corrected linestrings', 0)
        return None 

### Polygons

In [698]:
def process_data_polygons(gdf):
    # Initialize the changes_made flag
    changes_made = False

    # Create an empty GeoDataFrame
    corrected_polygon_gdf = gpd.GeoDataFrame(columns=['geometry_corrected', 'id_lokalid', 'tabel', 'old_Z', 'idx'])

    # Check if the GeoDataFrame contains Polygon or MultiPolygon geometry types
    if 'Polygon' in gdf['geometry'].geom_type.unique():
        # Iterate through each row in the GeoDataFrame
        for idx, row in gdf.iterrows():
            geometry = row['geometry']

            # Check if the geometry type is 'Polygon'
            if geometry.geom_type == 'Polygon':
                # Get a list of coordinates from the exterior of the Polygon geometry
                coords_list = list(geometry.exterior.coords)

                z_value_list = []
                

                if any(-999 in coord for coord in coords_list):
                    # Iterate over each coordinate in the list
                    for i, (x, y, z) in enumerate(coords_list):
                        # Check if the Z value of the coordinate is greater than 5000
                        if z == -999:
                            z_value_list.append((i, z))
                            
                            if gdf['tabel'][0] in DTM_objects:
                            
                            
                                # Replace the Z value with a new value obtained from Load_DHM function
                                new_z_value = Load_DTM(Kommune_kode, x, y, row['id_lokalid'], row['tabel'])
                            
                            else:
                                
                                # Replace the Z value with a new value obtained from Load_DHM function
                                new_z_value = Load_DHM(Kommune_kode, x, y, row['id_lokalid'], row['tabel'])
                                
                            #insert Coordinates into the coord_list at the right place using index 
                            coords_list[i] = (x, y, new_z_value)
                            
                            changes_made = True  # Set the flag to True to indicate changes were made

                    # Create a new Polygon with the modified exterior coordinates
                    modified_polygon = Polygon(coords_list)

                    # Create a dictionary for the data of the new row
                    data = {
                        'geometry_corrected': [modified_polygon],
                        'id_lokalid': [row['id_lokalid']],
                        'tabel': [row['tabel']],
                        'old_Z': [z_value_list],
                        'idx': [idx]
                    }

                    # Concatenate the new data to the corrected_polygon_gdf GeoDataFrame
                    corrected_polygon_gdf = pd.concat([corrected_polygon_gdf, pd.DataFrame(data)], ignore_index=True)

    elif 'MultiPolygon' in gdf['geometry'].geom_type.unique():
        print('_________ MultiPolygons NOT ACCEPTED __________')
        return None

    # Check if any changes were made
    if changes_made:
        print('corrected polygons', len(corrected_polygon_gdf))
        
        gdf = pd.merge(gdf, corrected_polygon_gdf, on=['tabel', 'id_lokalid'], how='inner')
        
        print('len gdf', len(gdf))
        
        return gdf
    else:
        print('corrected polygons', 0)
        return None

## Raster Layer Functions

### DHM Layer Function

In [636]:
def Load_DHM(Kommune_kode, x, y, id_lokalid, tabel):
    # Load DHM, from where the DHM interacts with the point
    

    # Ny SQL som udleder DHM fra databasen baseret på det punkt som skal have rettet sin kote
    query_DHM = f"""SELECT STI FROM DHM_OVERFLADE_NOHIST.DHM_OVERFLADE_NOHIST b
    WHERE MDSYS.SDO_RELATE(b.GEOMETRI, SDO_GEOMETRY('POINT({x} {y})', 25832), 'mask=ANYINTERACT') = 'TRUE'"""
    
    DHM_sti = pd.read_sql_query(con=connection.connect(), sql=sql_text(query_DHM))
    
    #print('Length of DHM STI', len(DHM_sti['sti']))
    
    for index in range(len(DHM_sti['sti'])):
        try:
            DHM_Raster = rasterio.open(DHM_sti['sti'][index])

            # Extract point value from raster
            row, col = DHM_Raster.index(x, y)
    
            # print("Point correspond to row, col: %d, %d"%(row,col))
    
            # print("Raster value on point %.2f \n"%DHM_Raster.read(1)[row,col])
        
            z = DHM_Raster.read(1)[row, col]
            
            z = round(z, 2)  # Round to two decimal places
        
            return z
    
        except Exception as e:
            # Handle exceptions (e.g., FileNotFoundError) and continue to the next index
            print(f"Failed to open STI at index {index}: {e}")
            continue
    
    # If none of the indices worked
    print("Unable to retrieve DHM value.")
    return -999 # return the old z value

### DTM Layer Function (TIL OBJEKTER SOM LIGGER I Terræn)

In [637]:
def Load_DTM(Kommune_kode, x, y, id_lokalid, tabel):
    # Load DHM, from where the DHM interacts with the point
    

    # Ny SQL som udleder DHM fra databasen baseret på det punkt som skal have rettet sin kote
    query_DTM = f"""SELECT STI FROM DHM_Terraen_NOHIST.DHM_Terraen_NOHIST b
    WHERE MDSYS.SDO_RELATE(b.GEOMETRI, SDO_GEOMETRY('POINT({x} {y})', 25832), 'mask=ANYINTERACT') = 'TRUE'"""
    
    DTM_sti = pd.read_sql_query(con=connection.connect(), sql=sql_text(query_DTM))
    
    #print('Length of DHM STI', len(DHM_sti['sti']))
    
    for index in range(len(DTM_sti['sti'])):
        try:
            DTM_Raster = rasterio.open(DTM_sti['sti'][index])

            # Extract point value from raster
            row, col = DTM_Raster.index(x, y)
    
            # print("Point correspond to row, col: %d, %d"%(row,col))
    
            # print("Raster value on point %.2f \n"%DHM_Raster.read(1)[row,col])
        
            z = DTM_Raster.read(1)[row, col]
            
            z = round(z, 2)  # Round to two decimal places
        
            return z
    
        except Exception as e:
            # Handle exceptions (e.g., FileNotFoundError) and continue to the next index
            print(f"Failed to open STI at index {index}: {e}")
            continue
    
    # If none of the indices worked
    print("Unable to retrieve DHM value.")
    return -999 # return the old z value

# Activate Functions

## Choose Municipality

In [654]:
# VÆLG KOMMUNE OG KOMMUNEKODE
Kommune = 'Jammerbugt'

Kommune_kode = '0849'

## Get all Objects from the Specific Municipality

In [707]:
liste_af_objekter = []

for tabel in Tabels['view_name']:
    
    if tabel not in ('KOMMUNEOMRAADE', 'OMRAADEPOLYGON'):
    
        print(tabel)
        
        

        query2 =  f"""SELECT SDO_UTIL.TO_WKTGEOMETRY(geometri) AS geometri2, t.* from GEODANMARK_60_NOHIST.{tabel} t
        WHERE  MDSYS.SDO_RELATE(GEOMETRI, (select geometri from GEODANMARK_60_B.KOMMUNEOMRAADE where kommunekode = {Kommune_kode} and registreringtil is null ) ,'mask=ANYINTERACT ') = 'TRUE' AND t.VERTIKALSTEDFAESTELSESMETODE in ('Fotogrammetri', 'Landmåling') And geometristatus != 'Foreløbig'
        """
        
        #print(query2)

        result_df = Run_query(query2, tabel)
        
        print(len(result_df))



        liste_af_objekter.append(result_df)


AFVANDINGSGROEFT
937
ANLAEGDIVERSE
55
BADEBAADEBRO
0
BASSIN
62
BEGRAVELSESOMRAADE
29
BROENDDAEKSEL
10071
BRUGSGRAENSE
43922
BYGNING
51726
BYGVAERK
1592
BYKERNE
0
CHIKANE
511
DAEMNING
27
DHMHESTESKO
0
DHMLINJE
49
DIGE
725
ERHVERV
61
FOTOINDEX
0
GARTNERI
5
HAVN
21
HEDE
422
HEGN
65841
HELLE
259
HISTORISKFLADE
0
HISTORISKLINJE
0
HISTORISKPUNKT
0
HOEFDE
3
HOEJBEBYGGELSE
1
HOEJSPAENDINGSLEDNING
17
JERNBANE
0
KRATBEVOKSNING
238
KYST
349
LAVBEBYGGELSE
3046
MAST
11680
NEDLOEBSRIST
18549
PARKERING
555
PARKERINGSOMRAADE
0
PLADS
0
RAASTOFOMRAADE
15
REKREATIVTOMRAADE
0
SANDKLIT
1
SKORSTEN
42
SKOV
1686
SKRAENT
1776
SOE
2033
SPORTSBANE
0
STARTBANE
0
STATUESTEN
18
SYSTEMLINJE
0
TEKNISKANLAEGFLADE
157
TEKNISKANLAEGPUNKT
12
TEKNISKAREAL
119
TELEMAST
53
TOGSTATION
0
TRAE
5871
TRAEGRUPPE
10786
TRAFIKHEGN
139
UDPEGNINGFLADE
0
UDPEGNINGLINJE
0
UDPEGNINGPUNKT
0
VAADOMRAADE
331
VANDAFSTROEMNINGSOPLAND
0
VANDHAENDELSE
0
VANDKNUDE
0
VANDLOEBSKANT
5167
VANDLOEBSMIDTE
19748
VEJKANT
22438
VEJMIDTE
34971
VINDMOELLE

# Run Data through functions

In [708]:
DTM_objects = ['AFVANDINGSGROEFT', 'BASSIN', 'BEGRAVELSESOMRAADE', 'BROENDDAEKSEL', 'CHIKANE', 'DIGE', 'HEDE', 'KYST', 'NEDLOEBSRIST', 'SOE', 'VAADOMRAADE', 'VANDLOEBSKANT', 'VANDLOEBSMIDTE', 'VEJKANT', 'VEJMIDTE']

# list to store the corrected dataframes
Corrected_data = []

for gdf in liste_af_objekter:
    
    if len(gdf) > 0:
    

        #if gdf['tabel'][0] == 'NEDLOEBSRIST' or gdf['tabel'][0] == 'UDPEGNINGLINJE' or gdf['tabel'][0] == 'BYGNING':

        # Check if gdf is a GeoDataFrame and has at least one row
        if isinstance(gdf, gpd.GeoDataFrame) and len(gdf) > 0:
            # Check if the geometry type is 'Point'
            if 'Point' in gdf['geometry'].geom_type.unique():
                print('-----', gdf['tabel'].unique(), '-----')
                print("GeoDataFrame Information:")
                print("Geometry Type:", gdf['geometry'].geom_type.unique())
                print("Number of Rows:", len(gdf))

                corrected_points = process_data_point(gdf)

                if corrected_points is not None: 


                    Corrected_data.append(corrected_points)

                    print("\n")

            elif 'LineString' in gdf['geometry'].geom_type.unique():
                print('-----', gdf['tabel'].unique(), '-----')
                print("GeoDataFrame Information:")
                print("Geometry Type:", gdf['geometry'].geom_type.unique())
                print("Number of Rows:", len(gdf))



                corrected_linestrings = process_data_linestrings(gdf)

                if corrected_linestrings is not None:

                    Corrected_data.append(corrected_linestrings)

                    print("\n")

            elif 'Polygon' in gdf['geometry'].geom_type.unique() and gdf['tabel'].unique() != ['FOTOINDEX']:
                print('-----', gdf['tabel'].unique(), '-----')
                print("GeoDataFrame Information:")
                print("Geometry Type:", gdf['geometry'].geom_type.unique())
                print("Number of Rows:", len(gdf))

                corrected_polygons = process_data_polygons(gdf)

                if corrected_polygons is not None:

                    Corrected_data.append(corrected_polygons)

                    print("\n")

----- ['AFVANDINGSGROEFT'] -----
GeoDataFrame Information:
Geometry Type: ['LineString']
Number of Rows: 937
corrected linestrings 0
----- ['ANLAEGDIVERSE'] -----
GeoDataFrame Information:
Geometry Type: ['LineString']
Number of Rows: 55
corrected linestrings 0
----- ['BASSIN'] -----
GeoDataFrame Information:
Geometry Type: ['Polygon']
Number of Rows: 62
corrected polygons 0
----- ['BEGRAVELSESOMRAADE'] -----
GeoDataFrame Information:
Geometry Type: ['Polygon']
Number of Rows: 29
corrected polygons 0
----- ['BROENDDAEKSEL'] -----
GeoDataFrame Information:
Geometry Type: ['Point']
Number of Rows: 10071
corrected points 0
----- ['BRUGSGRAENSE'] -----
GeoDataFrame Information:
Geometry Type: ['LineString']
Number of Rows: 43922
corrected linestrings 0
----- ['BYGNING'] -----
GeoDataFrame Information:
Geometry Type: ['Polygon']
Number of Rows: 51726
corrected polygons 9
len gdf 9


----- ['BYGVAERK'] -----
GeoDataFrame Information:
Geometry Type: ['LineString']
Number of Rows: 1592
correct

# OUTPUT TIL GEODANMARK KLIENTEN
## Skal ud i en FilGeodatabase

In [709]:
# Drop columns


Corrected_data2 = []

for gdf in Corrected_data:
    print(gdf.tabel[0])
    
    
    # List of column names to drop
    columns_to_drop = ['geometri']
    
    
    if 'geometri' in gdf.columns:
        gdf = gdf.drop(columns=columns_to_drop)
    
    # Rename 'geometry_corrected' to 'geometri' if it exists
    if 'geometry_corrected' in gdf.columns:
        gdf = gdf.rename(columns={'geometry_corrected': 'geometri'})
    
    # Set the 'geometry' column as the active geometry column if 'geometri' exists
    if 'geometri' in gdf.columns:
        gdf = gdf.set_geometry('geometri')
    
    columns_to_drop_2 = ['gmlid', 'old_Z', 'idx', 'objectid', 'geometry_x', 'geometry_y', 'geometry', 'geometri2']
    
    for column in columns_to_drop_2:
        if column in gdf.columns:
            gdf = gdf.drop(columns=[(column)])
            
    Corrected_data2.append(gdf)

BYGNING
HEGN
VANDLOEBSMIDTE
VEJKANT
VEJMIDTE


In [710]:
def AttributeManager(gdf):
    
    """Changes and Adds Attributes to the Dataframe"""
    
    gdf['action'] = 'update'
    
    gdf['f_proces'] = 'Maskinel fejlrettelse'
    
    gdf['kommentar'] = 'Test'
    
    return gdf

In [714]:
from datetime import datetime, timedelta

def TimeManager(gdf):
    
    # Virkning Fra
    gdf['virk_fra'] = '2023-10-23T13:00:00.0000000+00:00'
    # Define the 'Europe/Copenhagen' time zone
    copenhagen_tz = pytz.timezone('Europe/Copenhagen')

    # Make sure 'reg_fra' is in UTC time zone
    if 'reg_fra' in gdf.columns:
        gdf['reg_fra'] = gdf['reg_fra'].apply(lambda x: x.tz_localize('UTC'))

        # Define the start and end dates for daylight saving time in Denmark
        gdf['dst_start'] = gdf['reg_fra'].apply(lambda x: copenhagen_tz.localize(datetime(x.year, 3, 25, 2, 0)))
        gdf['dst_end'] = gdf['reg_fra'].apply(lambda x: copenhagen_tz.localize(datetime(x.year, 10, 28, 2, 0)))

        # Adjust the time based on daylight saving time rules (sommertid eller vintertid)
        gdf['reg_fra'] = gdf.apply(lambda row: row['reg_fra'] - timedelta(hours=2) if row['dst_start'] <= row['reg_fra'] < row['dst_end'] else row['reg_fra'] - timedelta(hours=1), axis=1)

        # Format the 'reg_fra' column into the desired format
        gdf['reg_fra'] = gdf['reg_fra'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f000000+00:00')

        # Drop the temporary 'dst_start' and 'dst_end' columns
        gdf = gdf.drop(columns=['dst_start', 'dst_end'])

    return gdf

In [712]:
# Link til forkortelser 
#https://geodanmark.nu/Spec6/HTML5/DK/StartHer.htm#GEDS6-DK/4.8%20Bilag%20H%20Forkortedenavne.htm%3FTocPath%3D4.0%2520BILAG%7C_____5

Corrected_data3 = []

for gdf in Corrected_data2:
    
    #if gdf['tabel'][0] == 'NEDLOEBSRIST' or gdf['tabel'][0] == 'UDPEGNINGLINJE' or gdf['tabel'][0] == 'BYGNING':
    
    print(gdf.tabel[0])

    column_mapping = {
    'tabel': 'objType',
    'geometristatus': 'g_status',
    'registreringsspecifikation': 'reg_spec',
    'forretningsomraade': 'f_omr',
    'forretningsproces': 'f_proces',
    'registreringsaktoer': 'reg_akt',
    'registreringfra': 'reg_fra',
    'registreringtil': 'reg_til',
    'virkningsaktoer': 'virk_akt',
    'virkningtil': 'virk_til',
    'plannoejagtighed': 'p_noej',
    'planstedfaestelsesmetode': 'p_smetode',
    'vertikalnoejagtighed': 'v_noej',
    'vertikalstedfaestelsesmetode': 'v_smetode',
    'applikation': 'app',
    'forretningshaendelse': 'f_haendels',
    'id_namespace' : 'id_namespa',
    'virkningfra' : 'virk_fra',
    'bygningstype' : 'bygningsty', 
    'maalestedbygning' : 'maalestedB',
    'underminimumbygning' : 'underMinim',
    'synligbygning' : 'synligBygn',
    'overlapbygning' : 'overlapByg',
    'metode3d' : 'metode3D',
    'bbruuid' : 'BBRUUID',
    'bbraktion' : 'BBRaktion',
    'brugesher': 'brugesHer', 
    'vejmyndighedsystem' : 'vejmyndigh',
    'cvfadmnr' : 'CVFadmnr',
    'kommunekode' : 'kommunekod',
    'synligvejkant' : 'synligVejk',
    'underminimumtelemast' : 'underMinim',
    'vandloebstype' : 'vandloebst',
    'faldretning': 'faldretnin', 
    'netværk' : 'netvaerk',
    'midtebredde' : 'midtebredd',
    'synligvandloebsmidte' : 'synligVand',
    'vanduuid' : 'vandUUID',
    'fraknude' : 'fraKnude',
    'tilknude' : 'tilKnude',
    'vejmidtetype' : 'vejmidtety',
    'vejmyndighed' : 'vejmyndigh',
    'vejkategori' : 'vejkategor',
    'tilogfrakoersel' : 'tilogfrako',
    'rundkoersel' : 'rundkoerse'
    
        
}

    # Rename columns and assign new columns in a single step
    gdf = gdf.rename(columns=column_mapping)

    #print(gdf.columns)

    gdf.crs = 'EPSG:25832'


    gdf = AttributeManager(gdf)

    gdf = TimeManager(gdf)
    
    print(len(gdf))

    #print(gdf)

    Corrected_data3.append(gdf)

BYGNING
9
HEGN
1
VANDLOEBSMIDTE
3
VEJKANT
1
VEJMIDTE
2


## Out

In [713]:
correct_daish_letters =  {
'Nedloebsrist': 'Nedloebsrist',
'Udpegningpunkt' : 'UdpegningPunkt',
'Udpegninglinje' : 'UdpegningLinje',
'Udpegningflade' : 'UdpegningFlade',
'Dhmhestesko'    : 'DHMHestesko',
'Dhmlinje'       : 'DHMLinje'
}

print(len(Corrected_data3))

# Specify the output file path (including the .gdb file)
output_file = 'C:/Users/B177999/Desktop/GeoDK/test2.gdb'

# Display the DataFrames with their names
for gdf in Corrected_data3:
        
    layer = gdf['objType'][0]
    
    print(layer)
    
    #if layer == 'DHMHESTESKO':
    
    columns_to_drop_3 = ['objType', 'geom_type', 'objtype']

    for column in columns_to_drop_3:
        if column in gdf.columns:
            gdf = gdf.drop(columns=[(column)])


    layer = layer.lower()

    layer = layer.capitalize()

    if layer in correct_daish_letters:

        layer = correct_daish_letters[layer]

    print(layer)
    print(gdf.columns)


    #if layer == 'Nedloebsrist':

    gdf.crs = CRS.from_epsg(25832)

    #print(gdf['reg_fra'])

    gdf.to_file(output_file, layer=layer, driver="OpenFileGDB")
        
# Create a ZIP file in the same directory as the .gdb file
zip_file_name = os.path.join(os.path.dirname(output_file), 'output')

shutil.make_archive(zip_file_name, 'zip', os.path.dirname(output_file), os.path.basename(output_file))

5
BYGNING
Bygning
Index(['id_namespa', 'id_lokalid', 'tempid', 'status', 'g_status', 'reg_spec',
       'dataansvar', 'f_haendels', 'f_omr', 'f_proces', 'reg_akt', 'reg_fra',
       'reg_til', 'virk_akt', 'virk_fra', 'virk_til', 'p_noej', 'p_smetode',
       'v_noej', 'v_smetode', 'app', 'kommentar', 'bygningsty', 'maalestedB',
       'metode3D', 'underMinim', 'BBRaktion', 'synligBygn', 'overlapByg',
       'BBRUUID', 'geometri', 'action'],
      dtype='object')
HEGN
Hegn
Index(['id_namespa', 'id_lokalid', 'tempid', 'status', 'g_status', 'reg_spec',
       'dataansvar', 'f_haendels', 'f_omr', 'f_proces', 'reg_akt', 'reg_fra',
       'reg_til', 'virk_akt', 'virk_fra', 'virk_til', 'p_noej', 'p_smetode',
       'v_noej', 'v_smetode', 'app', 'kommentar', 'hegnstype', 'geometri',
       'action'],
      dtype='object')
VANDLOEBSMIDTE
Vandloebsmidte
Index(['id_namespa', 'id_lokalid', 'tempid', 'status', 'g_status', 'reg_spec',
       'dataansvar', 'f_haendels', 'f_omr', 'f_proces', 'reg_akt'

'C:\\Users\\B177999\\Desktop\\GeoDK\\output.zip'